# Chapter 10: Using Gradio for applications

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1. **Build Basic UIs** - Create simple Gradio applications using the Interface class
2. **Configure Applications** - Set up flagging, authentication, and server options
3. **Work with Widgets** - Use Textbox, Audio, Image, and selection components
4. **Create Layouts** - Organize multiple interfaces using TabbedInterface
5. **Build Chatbot UIs** - Create interactive chatbot interfaces using Blocks
6. **Deploy Applications** - Share and deploy to Hugging Face Spaces

---

## 📖 Table of Contents

1. [Introduction to Gradio](#1-introduction-to-gradio)
2. [Building Your First Gradio App](#2-building-your-first-gradio-app)
3. [Configuring Flagging Options](#3-configuring-flagging-options)
4. [Authentication and Server Settings](#4-authentication-and-server-settings)
5. [Working with Widgets](#5-working-with-widgets)
6. [Selection Components](#6-selection-components)
7. [Tabbed Interfaces](#7-tabbed-interfaces)
8. [Creating Chatbot UIs](#8-creating-chatbot-uis)
9. [Deployment to Hugging Face Spaces](#9-deployment-to-hugging-face-spaces)
10. [Summary](#10-summary)

---

## 🔧 Environment Setup

First, let's install Gradio and other required packages.

In [ ]:
# # Install required packages
# !pip install -q gradio numpy scikit-image pillow

In [ ]:
# Import libraries
import gradio as gr
import numpy as np

print(f"✅ Gradio version: {gr.__version__}")

---

## 1. Introduction to Gradio <a id="1-introduction-to-gradio"></a>

### 🌟 What is Gradio?

**Gradio** is an open-source Python library that makes it quick and easy to build web-based demos for your machine learning models or any Python function.

### Key Benefits:

| Feature | Description |
|---------|-------------|
| **Easy to Use** | Create UIs with just a few lines of code |
| **No Frontend Skills** | No HTML, CSS, or JavaScript needed |
| **Shareable** | Generate public links to share with anyone |
| **Deployable** | Host for free on Hugging Face Spaces |

### Architecture Overview:

```
┌─────────────┐     ┌──────────────┐     ┌─────────────┐
│   Inputs    │ --> │   Function   │ --> │   Outputs   │
│ (Text/Image)│     │ (Your Logic) │     │  (Results)  │
└─────────────┘     └──────────────┘     └─────────────┘
```

---

## 2. Building Your First Gradio App <a id="2-building-your-first-gradio-app"></a>

### 🚀 The Interface Class

The `Interface` class is Gradio's main high-level class for creating simple UIs.

**Key Parameters:**
- `fn` - The function to wrap with the UI
- `title` - Title displayed at the top
- `inputs` - Input component(s)
- `outputs` - Output component(s)

In [ ]:
# Example 1: Simple Greeting App
import gradio as gr

def create_greeting(name):
    """Create a personalized greeting message."""
    if not name:
        return "Please enter your name!"
    return f"🎉 Welcome, {name}! Great to have you here!"

# Create the interface
greeting_app = gr.Interface(
    fn=create_greeting,
    title="✨ Welcome Greeter",
    inputs="text",
    outputs="text"
)

# Launch the app
greeting_app.launch(share=True)

In [ ]:
# Example 2: Word Counter App
import gradio as gr

def analyze_text(text):
    """Analyze text and return statistics."""
    if not text:
        return "No text provided"
    
    words = len(text.split())
    chars = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""📊 Text Analysis:
    
📝 Words: {words}
🔤 Characters: {chars}
📃 Sentences: {sentences}
📏 Avg words/sentence: {words/max(sentences,1):.1f}"""

text_analyzer = gr.Interface(
    fn=analyze_text,
    title="📊 Text Analyzer",
    inputs="text",
    outputs="text"
)

text_analyzer.launch(share=True)

In [ ]:
# Create interface with larger textboxes
text_analyzer = gr.Interface(
    fn=analyze_text,
    title="📊 Text Analyzer",
    inputs=gr.Textbox(
        label="Enter your text",
        placeholder="Paste or type your text here...",
        lines=8,  # Increased from default
        max_lines=15  # Can expand further
    ),
    outputs=gr.Textbox(
        label="Analysis Results",
        lines=6,  # Larger output box
        interactive=False  # Read-only
    ),
)

text_analyzer.launch(share=True)

---

## 3. Configuring Flagging Options <a id="3-configuring-flagging-options"></a>

### 🚩 What is Flagging?

Flagging allows users to mark outputs that need attention - useful for collecting feedback on model outputs.

By default, clicking "Flag" logs inputs, outputs, and timestamps to a CSV file in `.gradio/flagged/`.

In [ ]:
# Example: Sentiment Classifier with Custom Flagging
import gradio as gr
import random

def classify_sentiment(text):
    """Simple mock sentiment classifier."""
    positive_words = ['good', 'great', 'love', 'excellent', 'happy', 'amazing']
    negative_words = ['bad', 'terrible', 'hate', 'awful', 'sad', 'horrible']
    
    text_lower = text.lower()
    pos_count = sum(1 for w in positive_words if w in text_lower)
    neg_count = sum(1 for w in negative_words if w in text_lower)
    
    if pos_count > neg_count:
        return "😊 Positive Sentiment"
    elif neg_count > pos_count:
        return "😞 Negative Sentiment"
    else:
        return "😐 Neutral Sentiment"

sentiment_app = gr.Interface(
    fn=classify_sentiment,
    title="💭 Sentiment Classifier",
    inputs="text",
    outputs="text",
    flagging_options=["Correct ✓", "Incorrect ✗", "Unsure ?"]
)

sentiment_app.launch(share=True)

---

## 4. Authentication and Server Settings <a id="4-authentication-and-server-settings"></a>

### 🔐 Adding Authentication

You can password-protect your Gradio app using the `auth` parameter.

### 🌐 Server Configuration

| Parameter | Default | Description |
|-----------|---------|-------------|
| `server_name` | "127.0.0.1" | IP address to bind to |
| `server_port` | 7860 | Port number |
| `share` | False | Create public link |

In [ ]:
# Example: Password-Protected Calculator
import gradio as gr

def calculate(num1, num2, operation):
    """Perform basic arithmetic operations."""
    operations = {
        "Add (+)": num1 + num2,
        "Subtract (-)": num1 - num2,
        "Multiply (×)": num1 * num2,
        "Divide (÷)": num1 / num2 if num2 != 0 else "Error: Division by zero"
    }
    result = operations.get(operation, "Invalid operation")
    return f"Result: {result}"

# Custom authentication function
def authenticate(username, password):
    # In production, use secure authentication!
    return username == "demo" and password == "demo123"

calculator = gr.Interface(
    fn=calculate,
    title="🔢 Secure Calculator",
    inputs=[
        gr.Number(label="First Number"),
        gr.Number(label="Second Number"),
        gr.Dropdown(["Add (+)", "Subtract (-)", "Multiply (×)", "Divide (÷)"], label="Operation")
    ],
    outputs="text"
)

# Launch with authentication
calculator.launch(auth=authenticate,share=True)

# Or use simple tuple: auth=("username", "password")
# calculator.launch(share=True)

In [ ]:
# Example: Sharing Options
import gradio as gr

def echo(text):
    return f"You said: {text}"

demo = gr.Interface(fn=echo, inputs="text", outputs="text", title="Echo App")

# Different launch options:

# Local only (default)
# demo.launch()

# Create a public shareable link (expires in 72 hours)
demo.launch(share=True)

# Make accessible on local network
# demo.launch(server_name="0.0.0.0", server_port=5000)

---

## 5. Working with Widgets <a id="5-working-with-widgets"></a>

### 📝 Textbox Component

Customize text inputs with labels, placeholders, and multi-line support.

In [ ]:
# Example: Story Generator with Custom Textbox
import gradio as gr

def generate_story(character, setting, mood):
    """Generate a short story based on inputs."""
    story = f"""📖 Once upon a time...

In the {setting}, there lived {character}. 
The atmosphere was {mood.lower()}, and adventure awaited around every corner.

One day, {character} discovered something extraordinary that would change everything...

[To be continued...]"""
    return story

story_gen = gr.Interface(
    fn=generate_story,
    title="📚 Story Starter Generator",
    inputs=[
        gr.Textbox(label="Main Character", placeholder="e.g., a brave knight", lines=1),
        gr.Textbox(label="Setting", placeholder="e.g., enchanted forest", lines=1),
        gr.Textbox(label="Mood", placeholder="e.g., mysterious, cheerful", lines=1)
    ],
    outputs=gr.Textbox(label="Your Story", lines=10)
)

story_gen.launch(share=True)

### 🎵 Audio Component

Work with audio files - upload or record from microphone.

In [ ]:
# Example: Audio Speed Modifier
import gradio as gr
import numpy as np

def modify_audio(audio, speed_factor):
    """Modify audio playback speed."""
    if audio is None:
        return None
    
    sample_rate, data = audio
    
    # Simple speed modification by resampling
    if speed_factor != 1.0:
        # Calculate new length
        new_length = int(len(data) / speed_factor)
        indices = np.linspace(0, len(data) - 1, new_length).astype(int)
        data = data[indices]
    
    return (sample_rate, data)

audio_app = gr.Interface(
    fn=modify_audio,
    title="🎵 Audio Speed Modifier",
    inputs=[
        gr.Audio(sources=["upload", "microphone"], type="numpy", label="Audio Input"),
        gr.Slider(minimum=0.5, maximum=2.0, value=1.0, step=0.1, label="Speed Factor")
    ],
    outputs=gr.Audio(label="Modified Audio")
)

audio_app.launch(share=True)

### 🖼️ Image Component

Process images with support for upload, webcam, and clipboard.

In [ ]:
# Example: Image Filter App
import gradio as gr
import numpy as np

def apply_filter(image, filter_type):
    """Apply various filters to an image."""
    if image is None:
        return None
    
    if filter_type == "Grayscale":
        # Convert to grayscale
        gray = np.mean(image, axis=2, keepdims=True)
        return np.repeat(gray, 3, axis=2).astype(np.uint8)
    
    elif filter_type == "Invert":
        return 255 - image
    
    elif filter_type == "Sepia":
        sepia_filter = np.array([[0.393, 0.769, 0.189],
                                  [0.349, 0.686, 0.168],
                                  [0.272, 0.534, 0.131]])
        sepia = image @ sepia_filter.T
        return np.clip(sepia, 0, 255).astype(np.uint8)
    
    elif filter_type == "Brighten":
        return np.clip(image.astype(np.int16) + 50, 0, 255).astype(np.uint8)
    
    return image

filter_app = gr.Interface(
    fn=apply_filter,
    title="🎨 Image Filter Studio",
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),
        gr.Dropdown(["Grayscale", "Invert", "Sepia", "Brighten"], value="Grayscale", label="Filter")
    ],
    outputs=gr.Image(label="Filtered Image")
)

filter_app.launch(share=True)

---

## 6. Selection Components <a id="6-selection-components"></a>

### 🎚️ Dropdown and Slider

Allow users to make selections from predefined options.

In [ ]:
# Example: Unit Converter
import gradio as gr

def convert_units(value, from_unit, to_unit):
    """Convert between different units."""
    # Conversion to meters (base unit)
    to_meters = {
        "Meters": 1,
        "Kilometers": 1000,
        "Centimeters": 0.01,
        "Miles": 1609.34,
        "Feet": 0.3048,
        "Inches": 0.0254
    }
    
    # Convert to meters, then to target unit
    meters = value * to_meters[from_unit]
    result = meters / to_meters[to_unit]
    
    return f"{value} {from_unit} = {result:.4f} {to_unit}"

units = ["Meters", "Kilometers", "Centimeters", "Miles", "Feet", "Inches"]

converter = gr.Interface(
    fn=convert_units,
    title="📏 Unit Converter",
    inputs=[
        gr.Number(label="Value", value=1),
        gr.Dropdown(units, value="Meters", label="From"),
        gr.Dropdown(units, value="Feet", label="To")
    ],
    outputs=gr.Textbox(label="Result")
)

converter.launch(share=True)

In [ ]:
# Example: Color Mixer with Sliders
import gradio as gr
import numpy as np

def mix_colors(red, green, blue):
    """Create a color swatch from RGB values."""
    # Create a 100x100 pixel image with the selected color
    color_swatch = np.zeros((100, 100, 3), dtype=np.uint8)
    color_swatch[:, :, 0] = red
    color_swatch[:, :, 1] = green
    color_swatch[:, :, 2] = blue
    
    hex_color = f"#{red:02x}{green:02x}{blue:02x}"
    return color_swatch, f"RGB({red}, {green}, {blue})\nHex: {hex_color.upper()}"

color_mixer = gr.Interface(
    fn=mix_colors,
    title="🎨 RGB Color Mixer",
    inputs=[
        gr.Slider(0, 255, value=128, step=1, label="🔴 Red"),
        gr.Slider(0, 255, value=128, step=1, label="🟢 Green"),
        gr.Slider(0, 255, value=128, step=1, label="🔵 Blue")
    ],
    outputs=[
        gr.Image(label="Color Preview"),
        gr.Textbox(label="Color Values")
    ]
)

color_mixer.launch(share=True)

---

## 7. Tabbed Interfaces <a id="7-tabbed-interfaces"></a>

### 📑 TabbedInterface Class

Combine multiple interfaces into a single application with tabs.

In [ ]:
# Example: Multi-Tool App with Tabs
import gradio as gr
import numpy as np

# Tool 1: Text Case Converter
def convert_case(text, case_type):
    if case_type == "UPPERCASE":
        return text.upper()
    elif case_type == "lowercase":
        return text.lower()
    elif case_type == "Title Case":
        return text.title()
    elif case_type == "Swap Case":
        return text.swapcase()
    return text

text_tool = gr.Interface(
    fn=convert_case,
    title="Text Case Converter",
    inputs=[
        gr.Textbox(label="Enter Text", lines=3),
        gr.Dropdown(["UPPERCASE", "lowercase", "Title Case", "Swap Case"], value="UPPERCASE", label="Convert To")
    ],
    outputs=gr.Textbox(label="Result", lines=3)
)

# Tool 2: Number Operations
def number_operations(numbers_str):
    try:
        numbers = [float(x.strip()) for x in numbers_str.split(',')]
        return f"""📊 Statistics:
Sum: {sum(numbers):.2f}
Average: {sum(numbers)/len(numbers):.2f}
Min: {min(numbers):.2f}
Max: {max(numbers):.2f}
Count: {len(numbers)}"""
    except:
        return "Please enter comma-separated numbers"

number_tool = gr.Interface(
    fn=number_operations,
    title="Number Statistics",
    inputs=gr.Textbox(label="Enter numbers (comma-separated)", placeholder="1, 2, 3, 4, 5"),
    outputs=gr.Textbox(label="Statistics")
)

# Tool 3: Random Generator
def generate_random(min_val, max_val, count):
    numbers = np.random.randint(int(min_val), int(max_val)+1, int(count))
    return ", ".join(map(str, numbers))

random_tool = gr.Interface(
    fn=generate_random,
    title="Random Number Generator",
    inputs=[
        gr.Number(label="Minimum", value=1),
        gr.Number(label="Maximum", value=100),
        gr.Slider(1, 20, value=5, step=1, label="How Many?")
    ],
    outputs=gr.Textbox(label="Random Numbers")
)

# Combine into tabbed interface
tabbed_app = gr.TabbedInterface(
    [text_tool, number_tool, random_tool],
    ["📝 Text", "🔢 Numbers", "🎲 Random"]
)

tabbed_app.launch(share=True)

---

## 8. Creating Chatbot UIs <a id="8-creating-chatbot-uis"></a>

### 💬 The Blocks Class

For more customized layouts (like chatbots), use the low-level `Blocks` API.

### Components for Chatbots:
- `gr.Chatbot` - Displays conversation history
- `gr.Textbox` - User input field
- `gr.Button` - Action buttons

In [ ]:
# Example: Simple FAQ Chatbot
import gradio as gr

# Simple knowledge base
FAQ_RESPONSES = {
    "hello": "Hello! 👋 How can I help you today?",
    "hi": "Hi there! 😊 What would you like to know?",
    "help": "I can answer questions about Gradio, Python, and general topics. Just ask!",
    "gradio": "Gradio is an open-source Python library for building ML demos and web apps with just a few lines of code!",
    "python": "Python is a versatile programming language, great for data science, web development, and AI!",
    "bye": "Goodbye! 👋 Have a great day!",
}

def get_response(message):
    """Generate a response based on keywords."""
    message_lower = message.lower()
    for keyword, response in FAQ_RESPONSES.items():
        if keyword in message_lower:
            return response
    return "I'm not sure about that. Try asking about 'gradio', 'python', or say 'help'!"

with gr.Blocks() as chatbot_app:
    gr.Markdown("# 🤖 FAQ Chatbot\nAsk me about Gradio, Python, or general topics!")
    
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    clear = gr.Button("🗑️ Clear Chat")
    
    def respond(message, chat_history):
        response = get_response(message)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": response})
        return "", chat_history
    
    def clear_chat():
        return []
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(clear_chat, None, chatbot)

chatbot_app.launch(share=True)

In [ ]:
# Example: Enhanced Chatbot with Persona
import gradio as gr
import random

personas = {
    "Friendly Assistant": {
        "greeting": "Hey there! 😊 I'm your friendly helper!",
        "style": lambda r: f"{r} 😊"
    },
    "Professional": {
        "greeting": "Good day. How may I assist you?",
        "style": lambda r: r
    },
    "Pirate": {
        "greeting": "Ahoy, matey! 🏴‍☠️ What brings ye here?",
        "style": lambda r: f"Arrr! {r}, matey! 🏴‍☠️"
    }
}

with gr.Blocks() as persona_chat:
    gr.Markdown("# 🎭 Persona Chatbot")
    
    with gr.Row():
        persona = gr.Dropdown(list(personas.keys()), value="Friendly Assistant", label="Select Persona")
    
    chatbot = gr.Chatbot(height=350)
    msg = gr.Textbox(placeholder="Say something...", label="")
    
    with gr.Row():
        clear = gr.Button("Clear")
        greet = gr.Button("Greet Me!")
    
    def chat(message, history, selected_persona):
        base_response = f"You said: '{message}'. That's interesting!"
        styled_response = personas[selected_persona]["style"](base_response)
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": styled_response})
        return "", history
    
    def greet_user(history, selected_persona):
        greeting = personas[selected_persona]["greeting"]
        history.append({"role": "assistant", "content": greeting})
        return history
    
    msg.submit(chat, [msg, chatbot, persona], [msg, chatbot])
    clear.click(lambda: [], None, chatbot)
    greet.click(greet_user, [chatbot, persona], chatbot)

persona_chat.launch(share=True)

In [ ]:
# close gradio servers
gr.close_all()

---

## 9. Deployment to Hugging Face Spaces <a id="9-deployment-to-hugging-face-spaces"></a>

### 🚀 Deploying Your Gradio App

Hugging Face Spaces provides **free hosting** for Gradio applications!

### Steps to Deploy:

1. **Create a directory** with your app file (e.g., `app.py`)

2. **Get a WRITE token** from https://huggingface.co/settings/tokens

3. **Deploy using terminal:**
   ```bash
   cd YourAppDirectory
   gradio deploy
   ```

4. **Follow the prompts** to configure your Space

### Example App File (`app.py`):

```python
import gradio as gr

def greet(name):
    return f"Hello, {name}!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()
```

### Sharing Options:

| Method | Duration | Use Case |
|--------|----------|----------|
| `share=True` | 72 hours | Quick demos |
| Hugging Face Spaces | Permanent | Production apps |

---

## 10. Summary <a id="10-summary"></a>

### 🎉 What We Learned

| Topic | Key Takeaway |
|-------|-------------|
| **Interface Class** | High-level API for simple apps with fn, inputs, outputs |
| **Flagging** | Collect user feedback on outputs |
| **Authentication** | Protect apps with username/password |
| **Widgets** | Textbox, Audio, Image components |
| **Selection** | Dropdown and Slider for user choices |
| **TabbedInterface** | Combine multiple interfaces |
| **Blocks** | Low-level API for custom layouts (chatbots) |
| **Deployment** | Free hosting on Hugging Face Spaces |

### 🔧 Quick Reference

```python
# Simple Interface
gr.Interface(fn=my_func, inputs="text", outputs="text").launch()

# With authentication
.launch(auth=("user", "pass"))

# Public sharing
.launch(share=True)

# Blocks for custom layouts
with gr.Blocks() as demo:
    # Add components
    demo.launch()
```

### 📚 Resources

- [Gradio Documentation](https://gradio.app/docs)
- [Hugging Face Spaces](https://huggingface.co/spaces)
- [Gradio GitHub](https://github.com/gradio-app/gradio)